In [2]:
import numpy as np
import pandas as pd
import os
import keras
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import regularizers
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import random
from keras.callbacks import ModelCheckpoint


def getPredictionDict() :
    prediction_dict = {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'H': 0, 'I': 0, 'K': 0, 'L': 0, 'M': 0, 
                   'N': 0, 'O': 0, 'P': 0, 'Q': 0, 'R': 0, 'S': 0, 'T': 0, 'U': 0, 'V': 0, 'W': 0, 'X': 0, 'Y': 0,
                      'SPACE' : 0, 'DEL' : 0, 'NOTHING' : 0}
    return prediction_dict


def checkIfRes(prediction_dict) :
    
    sum_of_frames = 0
    max_char = 'none'
    max_frame = 0
    
    for i in prediction_dict :
        sum_of_frames += prediction_dict[i]
        if max_frame < prediction_dict[i] :
            max_frame = prediction_dict[i]
            max_char = i
        
    if sum_of_frames < 60 :
        return "null", False
    
    if max_frame >= sum_of_frames//2 :
        return max_char, True
    
    return max_char, False


model = keras.models.load_model('model.weights.best.hdf5')
CATEGORIES = ['A','B','C','D','E','F','G','H','I','K','L','M','N','O','P','Q','R','S','T','U','V','W','X',
              'Y','SPACE','DEL','NOTHING']


catagories = np.array(CATEGORIES)
result_string = ""
prediction_dict = getPredictionDict()


import cv2, PySimpleGUI as sg


string_and_char_column = [
    [
        sg.Text("Character - ",size = (20, 1), font=("Helvetica" ,20), text_color = '#000000', background_color = '#96C8C3', pad = [0,5]),
        
    ],
    [
        sg.Text(size = (20, 3), key = 'char', font=("Helvetica", 20), text_color = '#000000', background_color = '#DFC9DA', pad = [0,10]),
        
    ],
    [
        sg.Text( size = (20, 2), font=("Helvetica"), text_color = '#000000', pad = [0,40]),
        
    ],
    [
        sg.Text("Text - ", size = (20, 1), font=("Helvetica", 20), text_color = '#000000', background_color = '#96C8C3', pad = [0,5]),
        
    ],
    [
        sg.Text(size = (20, 3), key = 'result', font=("Helvetica", 20), text_color = '#000000', background_color = '#DFC9DA', pad = [0,10]),
        
    ],
]

image_column = [
    [
        sg.Text("Video - ",size = (87, 1), font=("Helvetica"), text_color = '#000000', background_color = '#96C8C3', justification='center'),
    ],
    [
        sg.Image(filename='', key='image'),
    ],
]

layout = [
    [
        sg.Column(string_and_char_column),
        sg.VSeperator(),
        sg.Column(image_column),
    ],
]

window = sg.Window('Exploratory Project', layout, location=(0,0))
sg.theme('LightGrey6')

cap = cv2.VideoCapture(0)       # Setup the camera as a capture device


while True:                     # The PSG "Event Loop"

    event, values = window.Read(timeout=20, timeout_key='timeout')      # get events for the window with 20ms max wait
    
    _, frame = cap.read()
    
    if event is None:  
        break                                            # if user closed window, quit
        
    
    frame = cv2.flip(frame, 1)
    
    n = frame.shape[0]
    m = frame.shape[1]
    # n = 720 
    # m = 1280 
    
    m1 = (31 * m)//64
    
    #cv2.rectangle(frame, (620, 40), (940, 360), (255, 255, 255), 3)
    #ROI = frame[ 40:360 , 620:940 ]
    
    
    cv2.rectangle(frame, (m1, 40), (m1+320, 360), (255, 255, 255), 3)
    ROI = frame[ 40:360 , (m1):(m1+320) ]
    
    roi = cv2.flip(ROI, 1)
    
    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(gray_roi, (64,64))
    
    X = np.array(img).reshape(-1,64,64,1)
    X = X.astype('float32')/255.0
    
    y_prob = model.predict(X) 
    y_classes = y_prob.argmax(axis=-1)
    y = catagories[y_classes]
    y = y[0]
    
    window['char'].Update(str(y))
    
    if str(y) == 'NOTHING' :
        prediction_dict = getPredictionDict()
        
    else :    
        prediction_dict[str(y)] += 1
        result_letter, is_res = checkIfRes(prediction_dict)
        if is_res :
            if result_letter == 'DEL' :
                if result_string != '' :
                    result_string = result_string[:-1]
                    
            elif result_letter == 'SPACE' :
                result_string += ' '
                
            else :
                result_string += str(result_letter)
                
            prediction_dict = getPredictionDict()
            result_letter = 'none'
            
        
    window['result'].Update(result_string)
    
    frame = cv2.resize(frame, (700,700))
    
    window.FindElement('image').Update(
        data=cv2.imencode('.png', frame)[1].tobytes()
    ) # Update image in window

    
window.close()
cap.release()